In [ ]:
#Justas Baniulis 2015956 cat car pizza

In [19]:
from google.colab import drive
drive.mount('/content/drive')

MessageError: ignored

In [ ]:
pip install oidv6

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.0/4.0 MB 53.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.6/10.6 MB 58.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.6/79.6 KB 10.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 630.1/630.1 KB 56.4 MB/s eta 0:00:00
  Attempting uninstall: rsa
    Found existing installation: rsa 4.9
    Uninstalling rsa-4.9:
      Successfully uninstalled rsa-4.9
  Attempting uninstall: PyYAML
    Found existing installation: PyYAML 6.0
    Uninstalling PyYAML-6.0:
      Successfully uninstalled PyYAML-6.0


In [ ]:
import shutil

#viska atsisiunciam ir nukeliam i drive kad siustis nereiktu kas kart

!oidv6 downloader --classes cat car pizza --type_data train --no_labels --limit 800 --dest_dir OIDv6/train
!oidv6 downloader --classes cat car pizza --type_data validation --no_labels --limit 150 --dest_dir OIDv6/validation
!oidv6 downloader --classes cat car pizza --type_data test --no_labels --limit 150 --dest_dir OIDv6/validation

target_folder = "/content/drive/MyDrive/OIDv6"
download_folder = "/content/OIDv6"

shutil.copytree(download_folder, target_folder)

In [7]:
from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive')
folder_test = '/content/drive/MyDrive/OIDv6/test'
folder_train = '/content/drive/MyDrive/OIDv6/train'
folder_validation = '/content/drive/MyDrive/OIDv6/validation'


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
import torch.nn.functional as F

# modelis
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()

        # convolutional layers
        self.conv1 = nn.Conv2d(3, 16, 3, padding=1)
        self.conv2 = nn.Conv2d(16, 32, 3, padding=1)
        self.conv3 = nn.Conv2d(32, 64, 3, padding=1)
        self.conv4 = nn.Conv2d(64, 128, 3, padding=1)
        
        # pooling layer
        self.pool = nn.MaxPool2d(2, 2)

        # fully connected layers
        self.fc1 = nn.Linear(128 * 14 * 14, 512)
        self.fc2 = nn.Linear(512, 256)
        self.fc3 = nn.Linear(256, 3)

        # randomly set output features to 0
        self.dropout = nn.Dropout(0.5)

    def forward(self, x):

        # downsample the output of the convolutional layers
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = self.pool(F.relu(self.conv3(x)))
        x = self.pool(F.relu(self.conv4(x)))

        # reshape
        x = x.view(-1, 128 * 14 * 14)

        # dropout of fully connected layers
        x = self.dropout(F.relu(self.fc1(x)))
        x = self.dropout(F.relu(self.fc2(x)))
        x = self.fc3(x)

        return x

model = Net()


In [16]:
# augmentacijos & transformacijos
data_transforms = {
    'train': transforms.Compose([
        transforms.RandomHorizontalFlip(),
        transforms.RandomRotation(20),
        transforms.ColorJitter(brightness=0.4, contrast=0.2, saturation=0.2, hue=0.1),
        transforms.RandomGrayscale(p=0.1),
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5])
    ]),
    'validation': transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5])
    ]),
    'test': transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5])
    ])
}

In [ ]:
# tranformuojam ir susirenkam nuotraukas
train_data = datasets.ImageFolder(folder_train, transform=data_transforms['train'])
validation_data = datasets.ImageFolder(folder_validation, transform=data_transforms['validation'])
test_data = datasets.ImageFolder(folder_test, transform=data_transforms['test'])

# loadinam
train_loader = DataLoader(train_data, batch_size=16, shuffle=True)
val_loader = DataLoader(validation_data, batch_size=16, shuffle=False)
test_loader = DataLoader(test_data, batch_size=16, shuffle=False)

In [12]:
import torch.optim as optim
from torch.optim.lr_scheduler import StepLR
import numpy as np

# gpu
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = Net().to(device)

# loss func & optimizer (to update the model parameters)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.0005)
#scheduler = StepLR(optimizer, step_size=10, gamma=0.1)

num_epochs = 25

for epoch in range(num_epochs):
    # training

    model.train()
    train_loss_values = np.array([], dtype = np.float32)
    train_corrects = 0

    for inputs, labels in train_loader:
        # i gpu duodam batch
        inputs, labels = inputs.to(device), labels.to(device)

        # gradientai nulinami
        optimizer.zero_grad()

        # grazinam
        outputs = model(inputs)

        # argmax suskaiciuojam
        _, preds = torch.max(outputs, 1)
        loss = criterion(outputs, labels)

        # sumuojam teisingus spejimus ir loss
        train_corrects += torch.sum(preds == labels.data)
        train_loss_values = np.append(train_loss_values, loss.cpu().detach().numpy())

        # atnaujinam parametrus
        loss.backward()
        optimizer.step()

        #gradientai nulinami
        optimizer.zero_grad()

    #scheduler.step()

    #progreso stebejimas
    epoch_loss = np.mean(train_loss_values)
    epoch_acc = train_corrects.double() / len(train_data)
    print(f"Epoch {epoch+1}/{num_epochs}")
    print(f"Trn Loss: {epoch_loss:.4f}, trn Acc: {epoch_acc:.4f}")

    # validuojame (nebus dropouts)
    model.eval()
    val_loss_values = np.array([], dtype = np.float32)
    val_corrects = 0

    with torch.no_grad():
        for inputs, labels in val_loader:
            # i gpu duodam batch
            inputs, labels = inputs.to(device), labels.to(device)

            # grazinam ir per argmax skaiciuojam
            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)
            loss = criterion(outputs, labels)

            # sumuojam teisingus spejimus ir loss
            val_loss_values = np.append(val_loss_values, loss.cpu().detach().numpy())
            val_corrects += torch.sum(preds == labels.data)

        epoch_loss = np.mean(val_loss_values)
        epoch_acc = val_corrects.double() / len(validation_data)
        print(f"Val Loss: {epoch_loss:.4f}, Val Acc: {epoch_acc:.4f}")

Tesla T4
Epoch 1/25
Trn Loss: 0.9430, trn Acc: 0.5329
Val Loss: 0.9976, Val Acc: 0.4579
Epoch 2/25
Trn Loss: 0.8144, trn Acc: 0.6604
Val Loss: 0.6294, Val Acc: 0.7816
Epoch 3/25
Trn Loss: 0.7482, trn Acc: 0.7075
Val Loss: 0.6579, Val Acc: 0.7184
Epoch 4/25
Trn Loss: 0.6976, trn Acc: 0.7358
Val Loss: 0.4683, Val Acc: 0.8368
Epoch 5/25
Trn Loss: 0.6558, trn Acc: 0.7458
Val Loss: 0.5346, Val Acc: 0.7816
Epoch 6/25
Trn Loss: 0.6355, trn Acc: 0.7571
Val Loss: 0.4402, Val Acc: 0.8368
Epoch 7/25
Trn Loss: 0.5968, trn Acc: 0.7646
Val Loss: 0.3442, Val Acc: 0.8789
Epoch 8/25
Trn Loss: 0.5996, trn Acc: 0.7721
Val Loss: 0.4079, Val Acc: 0.8500
Epoch 9/25
Trn Loss: 0.5497, trn Acc: 0.7892
Val Loss: 0.3597, Val Acc: 0.8579
Epoch 10/25
Trn Loss: 0.5569, trn Acc: 0.7888
Val Loss: 0.3905, Val Acc: 0.8579
Epoch 11/25
Trn Loss: 0.5354, trn Acc: 0.8013
Val Loss: 0.3684, Val Acc: 0.8842
Epoch 12/25
Trn Loss: 0.4883, trn Acc: 0.8175
Val Loss: 0.3753, Val Acc: 0.8474
Epoch 13/25
Trn Loss: 0.5012, trn Acc: 0

In [61]:
# Test
model.eval()
y_true = []
y_pred = []

for inputs, labels in test_loader:
    # dedam i modeli
    inputs, labels = inputs.to(device), labels.to(device)

    # gaunam spejimus
    with torch.no_grad():
        outputs = model(inputs)
        _, preds = torch.max(outputs, 1)
        loss = criterion(outputs, labels)

    # kaupiam spejima ir tikrove
    y_true.extend(labels.cpu().numpy())
    y_pred.extend(preds.cpu().numpy())

In [18]:
# klasiu vardai
data_dir = '/content/drive/MyDrive/OIDv6/validation'
dataset = datasets.ImageFolder(data_dir)

class_names = dataset.classes

print(class_names)

FileNotFoundError: ignored

In [67]:
from sklearn.metrics import confusion_matrix, accuracy_score, precision_recall_fscore_support
import pandas as pd

# Metrikos
conf_matrix = confusion_matrix(y_true, y_pred)
accuracy = accuracy_score(y_true, y_pred)
precision, recall, f1_score, _ = precision_recall_fscore_support(y_true, y_pred, average='weighted')

conf_matrix_df = pd.DataFrame(conf_matrix, columns=class_names, index=class_names)

print("Confusion Matrix:")
print(conf_matrix_df)

print(f'Accuracy: {accuracy:.4f}')
print(f'Precision: {precision:.4f}')
print(f'Recall: {recall:.4f}')
print(f'F1 Score:: {f1_score:.4f}')

Confusion Matrix:
       car  cat  pizza
car    139    5      6
cat      7  130     13
pizza    2    2    146
Accuracy: 0.9222
Precision: 0.9243
Recall: 0.9222
F1 Score:: 0.9219


In [69]:
# modelio issisaugojimas
torch.save(model.state_dict(), "my_model2.pth")

In [12]:
# modelio uzsikrovimas
model = Net()
model_weights = torch.load('/content/my_model.pth', map_location=torch.device('cpu'))
model.load_state_dict(model_weights)

<All keys matched successfully>

In [1]:
!pip install flask-cors pyngrok


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 761.3/761.3 KB 11.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyngrok: filename=pyngrok-5.2.1-py3-none-any.whl size=19790 sha256=cb791ef45a7a92c6d5c9a67ccf3acfa4328d37ec3b71fa59c1dce40c4a4921d2
  Stored in directory: /root/.cache/pip/wheels/f6/89/59/49d4249e00957e94813ac136a335d10ed2e09a856c5096f95c
Successfully built pyngrok


In [20]:
# REST api
from flask import Flask, request, jsonify
from pyngrok import ngrok
import io
from PIL import Image
import base64
from flask_cors import CORS

app = Flask(__name__)
CORS(app, resources={r"*": {"origins": "*"}})
model = model.to('cpu')
model.eval()

@app.route("/test", methods=["GET"])
def test():
    return "This is a test endpoint."

def preprocess_image(image_file):
    image = Image.open(image_file)
    transformed_image = data_transforms['test'](image)
    return transformed_image.unsqueeze(0)

@app.route("/predict", methods=["POST"])
def predict():
    print("Received request data:", request.files, request.data)

    try:
        # Get the image from the request
        file = request.files.get("image")
        if not file:
            return jsonify({"error": "No image provided"}), 400

        # Preprocess the image
        processed_image = preprocess_image(file)
        processed_image = processed_image.to('cpu')

        # Make a prediction using the model
        with torch.no_grad():
            output = model(processed_image)
            _, predicted = torch.max(output.data, 1)
            prediction = predicted.item()

        # Prepare the response
        response = {
            "prediction": class_names[prediction]
        }
    except Exception as e:
        print("Error processing request:", e)
        return jsonify({"error": str(e)}), 500

    return jsonify(response)

# Set up pyngrok
ngrok_tunnel = ngrok.connect(5000)
print("Public URL:", ngrok_tunnel.public_url)

# Run the Flask app
app.run(port=5000)

Public URL: http://e16a-34-74-108-45.ngrok.io
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [04/Apr/2023 20:02:26] "POST /predict HTTP/1.1" 200 -


Received request data: ImmutableMultiDict([('image', <FileStorage: '338681621_796162218084476_2355268349714035306_n.jpg' ('image/jpeg')>)]) b''
